In [1]:
# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch
import sys, random
from transformers import (
    BertTokenizer,
    BertConfig,
    EarlyStoppingCallback,
    RobertaTokenizer,
    RobertaConfig,
    Trainer,
    TrainingArguments,
)
from collections import OrderedDict
import pandas as pd
# from transformers.adapters import AdapterArguments,setup_adapter_training
from datasets import load_from_disk, Dataset
from typing import Dict, List, Optional
# from icecream import ic

## path to text contrastive learning library
PATH_TO_COTRALIB = 'textContraLib'
PATH_TO_SENTEVAL = 'SentEval/'
PATH_TO_DATA = 'SentEval/data'

sys.path.insert(0, PATH_TO_COTRALIB)
sys.path.insert(0, PATH_TO_SENTEVAL)
import senteval
from models import (
    SimCSEModel,
    PromptModel,
    VisualGuidedPromptModel,
    VisualGuidedPromptModel_roberta,
)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from tools import print_table, print_full_table, print_trainable_parameters, evalTransferModel

In [2]:
model_name = 'bert-base-uncased'

# for fast loading
# visual_ft_path = 'data/flickr30k/flickr30k_trainset_img_features_768x768_p32.pt'

# visual_ft_path = 'data/COCO2017/COCO2017_trainset_img_features_1024x1024_p14.pt'
# visual_ft_path = 'data/COCO2017/COCO2017_trainset_img_features_768x768_p32.pt'
# visual_ft_path = 'data/flickr30k/flickr30k_trainset_img_features_768x768_p16.pt'
visual_ft_path = 'data/flickr30k/flickr30k_trainset_img_features_512_clip-vit-base-patch32.pt'
# visual_ft_path = 'data/flickr30k/flickr30k_trainset_img_features_512x512_p16.pt'
# visual_ft_path = 'data/COCO2017/COCO2017_trainset_img_features_clip-vit-base-patch32.pt'
# text_ft_path = 'data/COCO2017/COCO2017_trainset_text_features_clip-vit-base-patch32.pt'

# For test5/6#
# pretrained_vit_path = 'saved_clip_model/clip-vit-base-p32-finetuned_proj_768x768/pytorch_model.bin'

tokenizer = BertTokenizer.from_pretrained(model_name, cache_dir='./cache')

date = '8_15'

output_path = 'RESULTS\\FINAL\\bert-base_flickr_img_768to512_p32_mlp1_test2'

# output_path = 'RESULTS\\bert_large_coco_1024_test2'
# output_path = 'RESULTS\\bert_base_coco_768_test2'
# output_path = 'RESULTS\\bert_base_coco_768_test2'

template_idx = 23

In [3]:
# # Roberta
# model_name = 'roberta-base'

# visual_ft_path = 'data/flickr30k/flickr30k_trainset_img_features_512_clip-vit-base-patch32.pt'
# # visual_ft_path = 'data/COCO2017/COCO2017_trainset_img_features_clip-vit-base-patch32.pt'
# # visual_ft_path = 'data/COCO2017/COCO2017_trainset_img_features_768x768_p32.pt'
# # visual_ft_path = 'data/COCO2017/COCO2017_trainset_img_features_1024x1024_p14.pt'

# # pretrained_vit_path = 'saved_clip_model/clip-vit-base-p32-finetuned_proj_768x768/pytorch_model.bin'

# tokenizer = RobertaTokenizer.from_pretrained(model_name, cache_dir='./cache')
# date = '7_8'

# output_path = 'RESULTS\\roberta_base_flickr_768_test2'
# # output_path = 'RESULTS/roberta-base_coco_512_p32_test2'
# # output_path = 'RESULTS/roberta-large_coco_512_clip-vit-base-patch32_test2'

# template_idx = 25

In [4]:
def add_prompt(s, idx=0):
    prompt = {
        0: "This sentence : \"[ [S] ]\" means [MASK] !",
        1: "This sentence : \"[[S]]\" means <mask> !",     # roberta
        2: "\"[[S]]\" is [MASK] !",                        # X is [MASK].
        3:"\"[[S]]\" means [MASK] !",                      # X means [MASK].
        4: "\"[[S]]\" represents [MASK] !",                # X represents [MASK].
        5: "\"[[S]]\" stands for [MASK] !",                # # X stands for [MASK].
        6: "Text : \"[[S]]\" means [MASK] !",              # Text X means [MASK].
        7: "Sentence : \"[[S]]\" means [MASK] !",          # Sentence X means [MASK].
        8: "Statement : \"[[S]]\" means [MASK] !",         # Statement X means [MASK].
        9: "The statement : \"[[S]]\" means [MASK] !",     # The statement X means [MASK].
        10: "The sentence : \"[[S]]\" means [MASK] !",     # The sentence X means [MASK].
        11: "This statement : \"[[S]]\" means [MASK] !",   # This statement X means [MASK].
        12: "This sentence : \"[[S]]\" means [MASK] !",    # This sentence X means [MASK].
        13: "This sentence : \" [S] \" means [MASK] !",    # BERT
        14: "This sentence of \" [S] \" means [MASK] !",   # BERT
        15: "This sentence \"[S]\" means [MASK] !",           # This sentence "X" means [MASK].
        16: "This sentence : \" [S] \" means [MASK] !",    # This sentence : "X" means [MASK].
        17: "This sentence [ [S] ] means [MASK] !",        # This sentence [X] means [MASK].
        18: "This sentence ( [S] ) means [MASK] !",        # This sentence (X) means [MASK].
        19: "This sentence { [S] } means [MASK] !",        # This sentence {X} means [MASK].
        20: "This sentence \"{ [S] }\" means [MASK] !",    # This sentence "{X}" means [MASK].
        21: "This sentence : \"{ [S] }\" means [MASK] !",  # This sentence : "{X}" means [MASK].
        # Test ending punctuation mark
        22: "This sentence : \" [S] \" means [MASK] !",  # BERT
        23: "This sentence of \" [S] \" means [MASK] !",   # BERT
        24: "This sentence : \' [S] \' means <mask> !",    # Roberta
        25: "The sentence : \' [S] \' means <mask> !",     # Roberta
    }
    return prompt[idx].replace('[S]', s)

In [5]:
# process wiki-text for train
# use pandas to read simCSE-wiki.txt
# download from https://huggingface.co/datasets/princeton-nlp/datasets-for-simcse/resolve/main/wiki1m_for_simcse.txt
wiki_text_file = 'data/wiki1m_for_simcse.txt'
wiki = pd.read_csv(wiki_text_file, sep = '\t', header = None)
wiki.columns = ['text']
# use Dataset.from_pandas to convert to dataset
wiki_dataset = Dataset.from_pandas(wiki, split= "train") # type: ignore


un_remove_set = {'?', '*', '#', '´', '’', '=', '…', '|', '~', '/', '‚', '¿', '–', '»', '-', '€', '‘', '"', '(', '•', '`', '$', ':', '[', '”', '%', '£', '<', '[UNK]', ';', '“', '@', '_', '{', '^', ',', '.', '!', '™', '&', ']', '>', '\\', "'", ')', '+', '—'}
if model_name == 'roberta-base':
    remove_set = {'Ġ.', 'Ġa', 'Ġthe', 'Ġin', 'a', 'Ġ, ', 'Ġis', 'Ġto', 'Ġof', 'Ġand', 'Ġon', 'Ġ\'', 's', '.', 'the', 'Ġman', '-', 'Ġwith', 'Ġfor', 'Ġat', 'Ġwoman', 'Ġare', 'Ġ"', 'Ġthat', 'Ġit', 'Ġdog', 'Ġsaid', 'Ġplaying', 'Ġwas', 'Ġas', 'Ġfrom', 'Ġ:', 'Ġyou', 'Ġan', 'i', 'Ġby'}
else:
    remove_set = {".", "a", "the", "in", ",", "is", "to", "of", "and", "'", "on", "man", "-", "s", "with", "for", "\"", "at", "##s", "woman", "are", "it", "two", "that", "you", "dog", "said", "playing", "i", "an", "as", "was", "from", ":", "by", "white"}


def prepare_features(examples):
    
    total = len(examples['text'])
    # total = batch_size
    
    # Avoid "None" fields 
    for idx in range(total):
        if examples['text'][idx] is None:
            examples['text'][idx] = " "

    # add prompt
    sentences = [add_prompt(s, idx=template_idx) for s in examples['text']]

    # set max_length here:
    sent_features = tokenizer(sentences, max_length=77, truncation=True, padding="max_length")

    return sent_features

train_dataset = wiki_dataset.map(prepare_features, batched=True, remove_columns=['text'])  # TIME Consuming, about 9 mins for bert, 4.5 mins for roberta

#### [Recommended] just load the pre-processed file ###
# wiki_for_sts = "data/wiki_promptRobertabase_t2_64_with_mask"
# wiki_for_sts = "data/wiki_promptBertbase_t2_64_with_mask/"
# train_dataset = load_from_disk(wiki_for_sts)

Map:   0%|          | 0/995447 [00:00<?, ? examples/s]

In [6]:
res = []
import re
# EvalModel
def evalModel_withMask(model,tokenizer, pooler = "cls_before_pooler"):

    tasks = ['STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark', 'SICKRelatedness']
    
    # params = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 5}
    # params['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
    #                             'tenacity': 3, 'epoch_size': 2}
    params = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 10}
    params['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 64,
                                 'tenacity': 5, 'epoch_size': 4}

    def prepare(params, samples):
        return

    def batcher(params, batch, max_length=None):
        global SE
        # Handle rare token encoding issues in the dataset
        if len(batch) >= 1 and len(batch[0]) >= 1 and isinstance(batch[0][0], bytes):
            batch = [[word.decode('utf-8') for word in s] for s in batch]

        ### add prompt
        sentences = [add_prompt(' '.join(s).replace('\n', ' ').replace('  ', ' ').strip().lower(), idx=template_idx) for s in batch]

        batch = tokenizer.batch_encode_plus(
            sentences,
            return_tensors='pt',
            padding=True,
            max_length=max_length,
            truncation=True
        )

        # Move to the correct device
        for k in batch:
            batch[k] = batch[k].to(device)

        # Get raw embeddings
        with torch.no_grad():
            ### get mask embeddings
            output,mask_embs = model(**batch, output_hidden_states=True, return_dict=True,sent_emb=True)


        mask_embs = mask_embs.cpu()
        return mask_embs
        
    results = {}
    
    for task in tasks:
        se = senteval.engine.SE(params, batcher, prepare)
        result = se.eval(task)
        results[task] = result
    # res.append(results)
    task_names = []
    scores = []
    aligns = []
    uniforms = []
    
    for task in tasks:
        task_names.append(task)
        if task in results:
            if task in ['STS12', 'STS13', 'STS14', 'STS15', 'STS16']:
                scores.append("%.2f" % (results[task]['all']['spearman']['all'] * 100))
                aligns.append("%.3f" % (results[task]['all']['align_loss']['all']))
                uniforms.append("%.3f" % (results[task]['all']['uniform_loss']['all']))
            else:
                scores.append("%.2f" % (results[task]['test']['spearman'].correlation * 100))  # for STSB and SICK.
                aligns.append("%.3f" % (results[task]['test']['align_loss']))
                uniforms.append("%.3f" % (results[task]['test']['uniform_loss']))
        else:
            scores.append("0.00")
    task_names.append("Avg.")
    scores.append("%.2f" % (sum([float(score) for score in scores]) / len(scores)))
    aligns.append("%.3f" % (sum([float(score) for score in aligns]) / len(aligns)))
    uniforms.append("%.3f" % (sum([float(score) for score in uniforms]) / len(uniforms)))
    ret = {'avg_score': sum([float(score) for score in scores])/len(scores), 'scores':scores, 'aligns': aligns, 'uniforms': uniforms}
    print_full_table(task_names, scores, aligns, uniforms)
    
    return ret

In [7]:
corrs = []
class ProSimCSETrainer(Trainer):
    def __init__(self,**paraments):
        super().__init__(**paraments)
        
        self.best_sts = 0.0
        self.best_pool_sts = 0.0
        self.flag = False

    def evaluate(
        self,
        eval_dataset: Optional[Dataset] = None,
        ignore_keys: Optional[List[str]] = None,
        metric_key_prefix: str = "eval",
        eval_senteval_transfer: bool = False,
    ) -> Dict[str, float]:
        global corrs
        
        # Set params for SentEval (fastmode)
        params = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 5}
        params['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
                                            'tenacity': 3, 'epoch_size': 2}

        self.model.eval()
        all_results = evalModel_withMask(self.model,tokenizer, pooler = 'cls_before_pooler')
        corrs.append(all_results['avg_score'])
        if all_results['avg_score'] > self.best_sts:
            self.best_sts = all_results['avg_score']
            self.save_model(self.args.output_dir+"\\best-model")
            if self.flag: # For documentation
                # If it is the first time, record it; otherwise, update the last row of data
                lines_avg = open(self.args.output_dir+'\\avg_score_'+date+'.txt', 'r').readlines()
                # now edit the last line of the list of lines
                lines_avg[-1] = str(all_results['avg_score'])+'\n'
                # now write the modified list back out to the file
                open(self.args.output_dir+'\\avg_score_'+date+'.txt', 'w+').writelines(lines_avg)
                lines_scores = open(self.args.output_dir+'\\scores_'+date+'.txt', 'r').readlines()
                lines_scores[-1] = str(all_results['scores'])+'\n'
                open(self.args.output_dir+'\\scores_'+date+'.txt', 'w+').writelines(lines_scores)
                lines_aligns = open(self.args.output_dir+'\\aligns_'+date+'.txt', 'r').readlines()
                lines_aligns[-1] = str(all_results['aligns'])+'\n'
                open(self.args.output_dir+'\\aligns_'+date+'.txt', 'w+').writelines(lines_aligns)
                lines_uniforms = open(self.args.output_dir+'\\uniforms_'+date+'.txt', 'r').readlines()
                lines_uniforms[-1] = str(all_results['uniforms'])+'\n'
                open(self.args.output_dir+'\\uniforms_'+date+'.txt', 'w+').writelines(lines_uniforms)
            else:
                with open(self.args.output_dir+'\\avg_score_'+date+'.txt','a+') as f:
                    f.writelines(str(all_results['avg_score'])+'\n')
                with open(self.args.output_dir+'\\scores_'+date+'.txt','a+') as f:
                    f.writelines(str(all_results['scores'])+'\n')
                with open(self.args.output_dir+'\\aligns_'+date+'.txt','a+') as f:
                    f.writelines(str(all_results['aligns'])+'\n')
                with open(self.args.output_dir+'\\uniforms_'+date+'.txt','a+') as f:
                    f.writelines(str(all_results['uniforms'])+'\n')

            self.flag = True

        self.model.train()
        print('Spearman before pooler:',all_results['avg_score'],'\nmax spearman ',self.best_sts)
        self.log({'Spearman before pooler:':all_results['avg_score']})

        return {'eval_acc':all_results['avg_score']}

In [8]:
# config = BertConfig.from_pretrained(model_name, cache_dir='./cache')

# config = RobertaConfig.from_pretrained(model_name, cache_dir='./cache')

# ### PromptModel
# model = PromptModel(config)
# model.bert_model = model.bert_model.from_pretrained('bert-base-uncased', cache_dir='./cache')

### For loading original bert model parameters
# pretrain_bert = BertModel(config).from_pretrained('bert-base-uncased', cache_dir='./cache')
# pretrain_bert_dict = pretrain_bert.state_dict()
# embedding_dict = OrderedDict()
# encoder_dict = OrderedDict()
# pooler_dict = OrderedDict()
# for k, v in pretrain_bert_dict.items():
#     if 'embeddings' in k:
#         embedding_dict[k[11:]] = v
#     if 'encoder' in k:
#         encoder_dict[k[8:]] = v
#     if 'pooler' in k:
#         pooler_dict[k[13:]] = v

### DirectBYOLSEModel
# config.update({'decay':0.95, 'cut_dim':100})
# model = DirectBYOLSEModel(config)
# # online init
# model.online_embedding.load_state_dict(embedding_dict)
# model.online_encoder.load_state_dict(encoder_dict)
# # target init
# model.prepare()
# non_optimizer_list = [model.target_embedding,model.target_encoder]
# for layer in non_optimizer_list:
#     for para in layer.parameters():
#         para.requires_grad = False

### SimCSE
# model = SimCSEModel(config)
# model.embedding.load_state_dict(embedding_dict)
# model.encoder.load_state_dict(encoder_dict)
# model.projector.dense.load_state_dict(pooler_dict)


### PromptModel_Adapter
# model = PromptModel_Adapter(config)
# model.bert_model = model.bert_model.from_pretrained('bert-base-uncased', cache_dir='./cache')
# # Setup adapters
# adapter_args = AdapterArguments(
#     train_adapter = True,
# )
# setup_adapter_training(model.bert_model, adapter_args, 'adapter')
# model = model.cuda()

### add Lora
# config = LoraConfig(
#     r=4, #attention heads， updating the Rank of a matrix
#     lora_alpha=16, #alpha scaling
#     # target_modules=["q_proj", "v_proj"],
#     lora_dropout=0.01,
#     bias="none",
#     task_type="SEQ_CLS",
#     inference_mode=False,
# )
# model = get_peft_model(model, config)

### VisualGuidedPromptModel
visual_pool = torch.load(visual_ft_path).cuda()

##########
# N,C = visual_pool1.shape
# index = torch.LongTensor(random.sample(range(N), round(N/2))).cuda()
# visual_pool1 = visual_pool1[index]
# visual_pool2 = torch.load(visual_ft_path2).cuda()
# N,C = visual_pool2.shape
# index = torch.LongTensor(random.sample(range(N), round(N/2))).cuda()
# visual_pool2 = visual_pool2[index]
# visual_pool = torch.vstack((visual_pool1, visual_pool2)).cuda()
##########

# text_pool =  torch.load(text_ft_path).cuda()
############### BERT ############################
config = BertConfig.from_pretrained(model_name, cache_dir='./cache')
model = VisualGuidedPromptModel(config=config, visual_pool=visual_pool, text_pool=None)
model.bert_model = model.bert_model.from_pretrained(model_name, cache_dir='./cache')
model.temp_denosie = False
############### Roberta ############################
# config = RobertaConfig.from_pretrained(model_name, cache_dir='./cache')
# model = VisualGuidedPromptModel_roberta(config=config, visual_pool=visual_pool)
# model.roberta_model = model.roberta_model.from_pretrained(model_name, cache_dir='./cache')

################################## Test5、Test6 ##################################
# pretrained_dict = torch.load(pretrained_vit_path, map_location=device)
# model_dict = model.visual_bert.state_dict()
# new_pretrained_dict = OrderedDict()
# for k, v in pretrained_dict.items():
#     if 'text_model' in k: 
#         name = k[11:]   # remove `text_model.`
#         if 'self_attn.q_proj' in k:
#             name = name.replace('self_attn.q_proj', 'attention.self.query')
#         elif 'self_attn.k_proj' in k:
#             name = name.replace('self_attn.k_proj', 'attention.self.key')
#         elif 'self_attn.v_proj' in k:
#             name = name.replace('self_attn.v_proj', 'attention.self.value')
#         elif 'self_attn.out_proj' in k:
#             name = name.replace('self_attn.k_proj', 'attention.output.dense')
#         elif 'layer_norm1' in k:
#             name = name.replace('layer_norm1', 'attention.output.LayerNorm')
#         elif 'mlp.fc1' in k:
#             name = name.replace('mlp.fc1', 'intermediate.dense')
#         elif 'mlp.fc2' in k:
#             name = name.replace('mlp.fc2', 'output.dense')
#         elif 'layer_norm2' in k:
#             name = name.replace('layer_norm2', 'output.LayerNorm')
#         new_pretrained_dict[name] = v

# pretrained_dict = {k: v for k, v in new_pretrained_dict.items() if k in model_dict}
# model_dict.update(pretrained_dict)
# model.visual_bert.load_state_dict(model_dict)
################################## Test5、Test6 ##################################

model.cuda()
print_trainable_parameters(model)
# check if paras needs to update
# for k,v in model.named_parameters():
#     print('{}: {}'.format(k, v.requires_grad))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


trainable params: 110860288 || all params: 110860288 || trainable%: 100.0


In [9]:
# trainer parameter
model.cuda()
args = TrainingArguments(
    output_dir = output_path, #Remember to change the output folder
    evaluation_strategy   = "steps",
    eval_steps            = 50,
    learning_rate         = 3e-6,
    num_train_epochs      = 2.0,
    weight_decay          = 1e-6,
    per_device_train_batch_size = 128,
    per_device_eval_batch_size  = 128,
    save_steps            = 1000,
    # label_smoothing_factor= 0.05,
    save_total_limit      = 1,
    load_best_model_at_end= True,
    metric_for_best_model = 'acc',
    report_to             = None,
)
# early_stopping = EarlyStoppingCallback(early_stopping_patience=5)
trainer = ProSimCSETrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    # callbacks=[early_stopping],
)

In [10]:
trainer.train()

d:\Conda_data\envs\py39\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/15554 [00:00<?, ?it/s]

+--------+--------+--------+--------+--------+--------------+-----------------+--------+
| STS12  | STS13  | STS14  | STS15  | STS16  | STSBenchmark | SICKRelatedness |  Avg.  |
+--------+--------+--------+--------+--------+--------------+-----------------+--------+
| 64.34  | 77.86  | 69.68  | 78.67  | 72.15  |    75.23     |      69.10      | 72.43  |
| 0.187  | 0.229  | 0.215  | 0.225  | 0.189  |    0.182     |      0.145      | 0.196  |
| -1.961 | -2.114 | -2.063 | -1.794 | -1.932 |    -1.969    |      -1.725     | -1.937 |
+--------+--------+--------+--------+--------+--------------+-----------------+--------+
Spearman before pooler: 72.4325 
max spearman  72.4325
{'Spearman before pooler:': 72.4325, 'epoch': 0.01}
+--------+--------+--------+--------+--------+--------------+-----------------+--------+
| STS12  | STS13  | STS14  | STS15  | STS16  | STSBenchmark | SICKRelatedness |  Avg.  |
+--------+--------+--------+--------+--------+--------------+-----------------+--------+
| 6

TrainOutput(global_step=15554, training_loss=1.0015314290283486, metrics={'train_runtime': 26700.7686, 'train_samples_per_second': 74.563, 'train_steps_per_second': 0.583, 'train_loss': 1.0015314290283486, 'epoch': 2.0})

In [11]:
checkpoint_path = output_path+'/best-model/'
tokenizer1 = BertTokenizer.from_pretrained(model_name, cache_dir='./cache')
# ############### BERT ############################
config = BertConfig.from_pretrained(checkpoint_path+'config.json', cache_dir='./cache')
model1 = VisualGuidedPromptModel(config=config, visual_pool=visual_pool, text_pool=None)
model1.temp_denosie = False

model1.load_state_dict(torch.load(checkpoint_path+'pytorch_model.bin'))

# 测试时不启用 BatchNormalization 和 Dropout
model1.eval().cuda()

print_trainable_parameters(model1)

results = evalTransferModel(model1,tokenizer1, pooler = 'cls_before_pooler')

with open(output_path+'\\downstream_'+date+'.txt','a+') as f:
    f.writelines(str(results)+'\n')

trainable params: 110860288 || all params: 110860288 || trainable%: 100.0
+-------+-------+-------+-------+-------+-------+-------+-------+
|   MR  |   CR  |  SUBJ |  MPQA |  SST2 |  TREC |  MRPC |  Avg. |
+-------+-------+-------+-------+-------+-------+-------+-------+
| 82.67 | 87.63 | 95.43 | 89.25 | 86.93 | 86.85 | 74.07 | 86.12 |
+-------+-------+-------+-------+-------+-------+-------+-------+


In [ ]:
trainer.save_model()

In [ ]:
# Test Best model on downstream tasks
# model_name = 'bert-base-uncased'
model_name = 'bert-large-uncased'

checkpoint_path = output_path+'/best-model/'
# checkpoint_path = 'RESULTS/bert_base_flickr30k_512_clip-vit-base-patch32_test2/best-model-78.58/'

# visual_ft_path = 'data/flickr30k/flickr30k_trainset_img_features_768x768_p32.pt'
visual_ft_path = 'data/flickr30k/flickr30k_trainset_text_features_512_clip-vit-base-patch32.pt'
# visual_ft_path = 'data/COCO2017/COCO2017_trainset_img_features_512x512_p32.pt'

tokenizer1 = BertTokenizer.from_pretrained(model_name, cache_dir='./cache')

# ############### BERT ############################
config = BertConfig.from_pretrained(checkpoint_path+'config.json', cache_dir='./cache')
visual_pool = torch.load(visual_ft_path).cuda()
model1 = VisualGuidedPromptModel(config=config, visual_pool=visual_pool)
model1.temp_denosie = False

model1.load_state_dict(torch.load(checkpoint_path+'pytorch_model.bin'))

# 测试时不启用 BatchNormalization 和 Dropout
model1.eval().cuda()

print_trainable_parameters(model1)

evalTransferModel(model1,tokenizer1, pooler = 'cls_before_pooler')

trainable params: 336191488 || all params: 336191488 || trainable%: 100.0
+-------+-------+-------+-------+-------+-------+-------+-------+
|   MR  |   CR  |  SUBJ |  MPQA |  SST2 |  TREC |  MRPC |  Avg. |
+-------+-------+-------+-------+-------+-------+-------+-------+
| 86.45 | 91.01 | 96.19 | 89.17 | 90.37 | 86.59 | 74.14 | 87.70 |
+-------+-------+-------+-------+-------+-------+-------+-------+


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('float64'), dtype('<U5')) -> None

In [ ]:
# Test Best model on downstream tasks
model_name = 'roberta-large'

# checkpoint_path = 'RESULTS/roberta-large_flickr_512_clip-vit-base-patch32_test2/best-model-80.74/'
checkpoint_path = 'RESULTS/roberta-large_coco_512_clip-vit-base-patch32_test2/best-model-80.99/'

# visual_ft_path = 'data/COCO2017/COCO2017_trainset_img_features_clip-vit-base-patch32.pt'
visual_ft_path = 'data/COCO2017/COCO2017_trainset_img_features_512x512_p32.pt'
# visual_ft_path = 'data/COCO2017/COCO2017_trainset_img_features_768x768_p32.pt'

tokenizer = RobertaTokenizer.from_pretrained(model_name, cache_dir='./cache')

############### Roberta ############################
config = RobertaConfig().from_pretrained(checkpoint_path)

model = VisualGuidedPromptModel_roberta(config=config, visual_pool=visual_ft_path)
model.roberta_model = model.roberta_model.from_pretrained(model_name, cache_dir='./cache')

model.load_state_dict(torch.load(checkpoint_path+'pytorch_model.bin'))

# 测试时不启用 BatchNormalization 和 Dropout
model.eval().cuda()

print_trainable_parameters(model)

evalTransferModel(model,tokenizer, pooler = 'cls_before_pooler')

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


trainable params: 357458944 || all params: 357458944 || trainable%: 100.0
+-------+------+-------+------+-------+-------+-------+-------+
|   MR  |  CR  |  SUBJ | MPQA |  SST2 |  TREC |  MRPC |  Avg. |
+-------+------+-------+------+-------+-------+-------+-------+
| 84.14 | 89.0 | 94.28 | 88.7 | 86.24 | 81.53 | 73.11 | 85.29 |
+-------+------+-------+------+-------+-------+-------+-------+


[84.14, 89.0, 94.28, 88.7, 86.24, 81.53, 73.11, '85.29']